This example demonstrates the use of d-flip-flops and registers.

In [1]:
import magma as m
m.set_mantle_target("ice40")

## DFF

To use a DFF we import that `mantle` circuit `DFF`.
Calling `DFF()` creates an instance of a DFF.

A sequential logic element like a DFF is very similar
to a combinational logic element like a full adder.
It has inputs and outputs.
The inputs and outputs are wired up in the same way as
a combinational circuit.

In [2]:
from loam.boards.icestick import IceStick
from mantle import DFF

icestick = IceStick()
icestick.Clock.on() # Need to turn on the clock for sequential logic
icestick.J1[0].input().on()
icestick.J3[0].output().on()

main = icestick.DefineMain()
dff = DFF()
m.wire( dff(main.J1), main.J3 )
m.EndDefine()

import lattice ice40
import lattice mantle40


Since a flip-flop is a sequential logic element,
it has a clock.
The clock generator is a peripheral on the FPGA.
We need to turn it on if we want to use the clock.
This creates a global clock signal on the FPGA.
Note that we did not need to wire the clock to the DFF;
`magma` automatically wires the  clock to the flip-flop's clock input.

Let's compile and build.

In [3]:
m.compile("build/dff", main)

compiling main


In [4]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif dff.blif' dff.v
arachne-pnr -q -d 1k -o dff.txt -p dff.pcf dff.blif 
icepack dff.txt dff.bin
iceprog dff.bin

/Users/hanrahan/git/magmathon/notebooks/tutorial/build


init..
cdone: low
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x15 0x00 0x26 0x06 0x06 0x15 0x9F 0x4E
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


If we inspect the compiled verilog, we see that our mantle `DFF` uses the `SB_DFF` ice40 primitive.

In [5]:
%cat build/dff.v

module main (input  J1, output  J3, input  CLKIN);
wire  inst0_Q;
SB_DFF inst0 (.C(CLKIN), .D(J1), .Q(inst0_Q));
assign J3 = inst0_Q;
endmodule



## Register

A register is simply an array of flip-flops.
To create an instance of a register, call `Register`
with the number of bits `n` in the register.

In [6]:
from loam.boards.icestick import IceStick
from mantle import Register

icestick = IceStick()
icestick.Clock.on() # Need to turn on the clock for sequential logic
for i in range(4):
    icestick.J1[i].input().on()
    icestick.J3[i].output().on()

main = icestick.DefineMain()
register4 = Register(4)
m.wire( register4(main.J1), main.J3 )
m.EndDefine()

Registers and DFFs are very similar to each other.
The only difference is that the input and output to a DFF
are `Bit` values,
whereas the inputs and the outputs to registers are `Bits(n)`.

In [7]:
m.compile("build/register4", main)

compiling Register4
compiling main


In [8]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif register4.blif' register4.v
arachne-pnr -q -d 1k -o register4.txt -p register4.pcf register4.blif 
icepack register4.txt register4.bin
iceprog register4.bin

/Users/hanrahan/git/magmathon/notebooks/tutorial/build


init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x15 0x00 0x26 0x06 0x06 0x15 0x9F 0x4E
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.


If we inspect the compiled verilog, we see that our register is a module that instances a set of `SB_DFF`s.

In [9]:
%cat build/register4.v

module Register4 (input [3:0] I, output [3:0] O, input  CLK);
wire  inst0_Q;
wire  inst1_Q;
wire  inst2_Q;
wire  inst3_Q;
SB_DFF inst0 (.C(CLK), .D(I[0]), .Q(inst0_Q));
SB_DFF inst1 (.C(CLK), .D(I[1]), .Q(inst1_Q));
SB_DFF inst2 (.C(CLK), .D(I[2]), .Q(inst2_Q));
SB_DFF inst3 (.C(CLK), .D(I[3]), .Q(inst3_Q));
assign O = {inst3_Q,inst2_Q,inst1_Q,inst0_Q};
endmodule

module main (input [3:0] J1, output [3:0] J3, input  CLKIN);
wire [3:0] inst0_O;
Register4 inst0 (.I(J1), .O(inst0_O), .CLK(CLKIN));
assign J3 = inst0_O;
endmodule



## Enables and Resets

There are other flip-flops and registers with clock enables
and reset inputs.
The flip-flop will only be enabled if its clock enable input is true.
And it will be reset to its initial value if reset is true.

To create registers with these additional inputs,
set the optional arguments `has_ce` and/or `has_reset`
when instancing the register.

In [10]:
icestick = IceStick()
icestick.Clock.on()
for i in range(4):
    icestick.J1[i].input().on()
    icestick.J3[i].output().on()
icestick.J1[4].input().on() # ce signal
icestick.J1[5].input().on() # reset signal

main = icestick.DefineMain()
register4 = Register(4, init=5, has_ce=True, has_reset=True )
m.wire( register4(main.J1[0:4], ce=main.J1[4], reset=main.J1[5]), main.J3)
m.EndDefine()

To wire the optional clock inputs, clock enable and reset,
use named arguments when you call the register with its inputs.
As a general rule, clock inputs are handled differently than
other inputs.

Compile, build, and upload.

In [11]:
m.compile("build/register4ce", main)

compiling Register4CER_0005
compiling main


In [12]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif register4ce.blif' register4ce.v
arachne-pnr -q -d 1k -o register4ce.txt -p register4ce.pcf register4ce.blif 
icepack register4ce.txt register4ce.bin
iceprog register4ce.bin

/Users/hanrahan/git/magmathon/notebooks/tutorial/build


init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x15 0x00 0x26 0x06 0x06 0x15 0x9F 0x4E
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.
